- 자연어 처리의 하위 분야인 감성 분석을 해볼 것
- 머신 러닝을 적용하여 글쓴이의 성향에 따라 문서를 분류하는 방법
- IMDb의 영화 리뷰 5만 개로 이루어진 데이터셋을 사용
- 각 리뷰가 긍정인지 부정인지 구별하는 예측기 구축
- 순서
    > 텍스트 데이터의 정제와 준비 <br>
    > 텍스트 문서로부터 특성 벡터 구축하기 <br>
    > 영화 리뷰를 긍정 또는 부정으로 분류하는 머신 러닝 모델 훈련 <br>
    > 외부 메모리 학습을 사용하여 대용량 텍스트 데이터셋 다루기 <br>
    > 문서를 카테고리롤 묶기 위해 문서의 토픽 추론하기

# 1.텍스트 처리용 IMDb 영화 리뷰 데이터 준비

- 긍정/부정으로 레이블되어 있는 영화 리뷰 5만개로 구성
- 긍정이란 별 여섯 개 이상을 받은 영화
- 부정은 별 다섯개 아래를 받은 영화

## 1.1.영화 리뷰 데이터셋 구하기

- [리뷰 데이터 셋 링크]('https://ai.stanford.edu/~amaas/data/sentiment/')
- `tar.gz` 형태는 아래의 파이썬 코드로 처리 가능
---
```python
import tarfile

with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()
```

## 1.2.영화 리뷰 데이터셋을 더 간편한 형태로 전처리

In [72]:
import pyprind
import pandas as pd
import os

# 'basepath'를 압축 해제된 영화 리뷰 데이터셋이 있는 디렉토리로 바꿈

basepath = './datasets/aclImdb_v1/aclImdb'

labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()

df.columns = ['review', 'sentiment']

C:\Users\yunjc\AppData\Local\Temp\ipykernel_33436\2094106499.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt, labels[l]]], ignore_index=True)
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:22


In [73]:
# 클래스 레이블이 순서대로 나열 되어 있으므로 np.random 모듈의 permutation 함수를 사용하여 섞음
# 나중에 로컬 디스크에서 데이터를 조금씩 읽어 훈련 데이터셋과 테스트 데이터셋을 나눌 때 좋음
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('./datasets/movie_data.csv', index=False, encoding='utf-8')

In [74]:
df = pd.read_csv('./datasets/movie_data.csv', encoding='utf-8')
# df.columns = ['review', 'sentiment']
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [17]:
df.shape

(8731, 2)

# 2.BoW 모델

- 텍스트를 수치 특성 벡터로 표현하는 BoW 모델
- BoW 모델의 아이디어는 다음과 같음
    > 1. 전체 문서에 대해 고유한 토큰, 예를 들어 단어로 이루어진 어휘 사전을 만듬 <br>
    > 2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터를 만듬
- 각 문서에 있는 고유한 단어는 BoW 어휘 사전에 있는 모든 단어의 일부분에 지나지 않으므로 특성 벡터는 대부분이 0으로 채워짐
- 그래서 이 특성 벡터를 희소하다고 함

## 2.2.단어를 특성 벡터로 변환

In [18]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining', 'The weather is sweet', 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [19]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


- 특성 벡터의 각 인덱스는 `CountVectorizer`의 어휘 사전 딕셔너리에 저장된 정수 값에 해당
- 특성 벡터의 값들을 단어 빈도라고 함
- 문서 $d$에 등장한 %t%의 횟수를 %tf(t,d)%와 같이 씀
- BoW 모델은 문장이나 문서에 등장하는 단어의 순서를 상관하지 않음
- 특성 벡터에 나타나는 단어 빈의 순서는 보통 어휘 사전의 알파벳 순서를 따름

**n-gram 모델**

- BoW 모델에 있는 아이템 시퀀스를 **1-gram** 모델 또는 **unigram** 모델이라고 함
- 어휘 사전에 있는 각 아이템 또는 토큰이 하나의 단어를 표현
- 일반화하면 NLP에서 연속된 아이템(단어, 문자, 기호)의 시퀀스를 n-gram이라고 함
- n-gram에 n에 어떤 값을 선택할지는 애플리케이션마다 다름
- 예를 들어 Kanaris 등은 3 or 4의 n-gram이 좋은 성능을 낸다고 밝힘
- 'the sun is shining'을 1-gram과 2-gram으로 표현하면 다음과 같음
    ```text
    1-gram: 'the', 'sun', 'is', 'shining'
    2-gram: 'the sun', 'sun is', 'is shining'
    ```

## 2.2.tf-idf를 사용하여 단어 적합성 평가

- 텍스트 데이터를 분석할 때 클래스 레이블이 다른 문서에 같은 단어들이 나타나는 경우가 있음
- 일반적으로 자주 등장하는 단어는 유용하거나 판별에 필요한 정보를 가지고 있지 않음
- 특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법인 tf-idf를 사용
- tf-idf는 단어 빈도와 역문서 빈도의 곱으로 정의됨
$$
tf-idf(t, d) = tf(t,d) \times idf(t,d)
$$
- idf는 다음과 같음(역문서 빈도)
$$ \begin{align*}
idf(t, d) = \log\frac{n_d}{1+df(d,t)} & \\
& n_d:\,전체\,문서의\,개수 \\
& df(d,t):\,단어\,t가\,포함된\,문서\,d의\,개수 (분모에\,상수\,1을\,추가하는\,것은\,선택)
\end{align*}$$
- 훈련 샘플에 한 번도 등장하지 않는 단어가 있을 경우 분모가 0이 되지 않게 하기 위해 분모에 상수 추가
- log는 문서 빈도가 낮을 때 역문서 빈도 값이 너무 커지지 않도록 만듦

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


- 'is'가 가장 많이 나타났기 대문에 단어 빈도가 가장 큼
- 동일한 특성 벡터를 tf-idf로 변환하면 단어 'is'는 비교적 작은 tf-idf를 가짐
- 이는 판별에 유용한 정보를 가지고 있지 않음을 뜻함
- 사이킷런에 구현된 역문서 빈도 공식은 아래와 같이 다름
$$
idf(t,d)=\log\frac{1+n_d}{1+df(d,t)}
$$
- 비슷하게 계산하는 tf-idf 또한 앞서 정의한 공식과 다름
$$
tf-idf(t,d)=tf(t,d) \times (idf(t,d)+1)
$$
- 1을 더한 이유는 모든 문서에 등장하는 단어가 가중치 0이 되는 것을 막기 위해서임
- tf-idf를 계산하기 전에 단어 빈도를 정규화하지만 `TfidfTransformer`클래스는 tf-idf를 직접 정규화함
- 기본적으로 L2 정규화를 수행
- 정규화되지 않은 특성 벡터 $v$를 L2-norm으로 나누면 길이가 1인 벡터가 반환됨
$$
v_{norm}=\frac{v}{\begin{Vmatrix} v \end{Vmatrix}_2}=\frac{v}{\sqrt{v_{1}^2+v_{2}^2+\cdots+v_{n}^2}}=\frac{v}{\left ( \sum_{i=1}^n v_{i}^2 \right )^{1/2}}
$$

## 2.3.텍스트 데이터 정제

In [21]:
# 첫 번째 문서에서 마지막 50개의 문자를 출력
# 책의 예제와 똑같은 문자열을 찾아보자

candidate = df.loc[df['review'].str.contains('is seven.<br /><br />Title')].index
target=0
for idx in candidate:
    if "Not Available" in df.loc[idx, 'review']:
        target = idx
        print(idx, df.loc[idx, 'review'][-50:])
        
df.loc[target, 'review'][-50:]

'keup end of things, but definitely worth watching.'

In [22]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower())+' '.join(emoticons).replace('-',''))
    return text

- 첫 번째 정규 표현식 <[^>]*>를 사용하여 영화 리뷰에서 모든 html 마크업을 삭제
- 다음으로 이모티콘 삭제하고 저장
- 마지막으로 텍스트를 솜누자로 바꾸고 [\W]+를 사용하여 텍스트에서 단어가 아닌 문자를 모두 제거

In [23]:
preprocessor(df.loc[target, 'review'][-50:])

'keup end of things but definitely worth watching '

In [24]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [25]:
df['reveiw'] = df['review'].apply(preprocessor)

## 2.4.문서를 토큰으로 나누기

In [26]:
def tokenizer(text):
    return text.split()
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

- 토큰화 방법 중에는 단어를 변하지 않는 기본 형태인 어간으로 바꾸는 어간 추출이라는 다른 방법이 있음
- 여러 가지 형태를 갖는 단어를 같은 어간으로 매핑할 수 있음
- 파이썬의 NLTK 패키지를 사용

In [27]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

**어간 추출 알고리즘**

- 포터 어간 추출 알고리즘은 가장 오래되고 간단한 어간 추출 알고리즘
- 인기있는 다른 어간 추출 알고리즘으로는 **스노우볼 어간 추출**, **랭커스터 어간 추출기**가 있음
- 둘 다 모두 포터 어간 추출기보다 빠르지만 랭커스터 어간 추출기는 포터 어간 추출기보다 훨씬 공격적인 것으로 알려져 있음
- 어간 추출은 실제 사용하지 않는 단어를 만들 수 있음
- 표제어 추출 기법은 표제어라는 (문법적으로 옳은) 각 단어의 표준 형태를 얻는 것이 목적
- 표제어 추출은 어간 추출에 비해 계산이 어렵고 비용이 많이 듬
- 실전에서는 어간 추출과 표제어 추출이 텍스트 분류 성능에 미치는 영향이 크지 않음

- 불용어 제거
- 불용어는 모든 종류의 텍스트에 아주 흔하게 등장하는 단어
- 문서의 종류를 구별하는 데 사용할 수 있는 정보가 없거나 아주 조금만 있음
- 불용어 제거는 tf-idf보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 더 유용
- tf-idf에는 자주 등장하는 단어의 가중치가 이미 낮추어져 있음
- `nltk.download`함수를 사용

In [28]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yunjc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [29]:
from nltk.corpus import stopwords

stop=stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 3.문서 분류를 위한 로지스틱 회귀 모델 훈련

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression as LGR
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param_grid = [
    {'vect__ngram_range': [(1,1)], 'vect__stop_words': [stop, None], 'vect__tokenizer': [tokenizer, tokenizer_porter], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]},
    {'vect__ngram_range': [(1,1)], 'vect__stop_words': [stop, None], 'vect__tokenizer': [tokenizer, tokenizer_porter],
     'vect__use_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}
]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LGR(solver='liblinear', random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

- `n_jobs=-1`로 지정하는게 웬만하면 좋음
- 일부 시스템에서는 멀티프로세싱을 위해 `n_jobs=-1`로 지정할 때, `tokenizer`와 `tokenizer_porter` 함수의 직렬화에 문제가 발생할 수 있음
- 이럴 경우 `[str.split]`으로 바꾸어 해결 (다만 어간 추출은 할 수 없게 됨)

- `CounterVetorizer`와 `TfidfTransformer`를 두 기능을 하나로 합친 `TfidfVectorizer`로 대체
- `param_grid`는 두 개의 매개변수 딕셔너리로 구성되어 있음
- 첫 번째 딕셔너리는 `TfidfVectorizer`의 기본 매개변수 셋팅(`use_idf=True, norm='l2`)을 사용하여 tf-idf를 계산
- 두 번째 딕셔너리는 단어 빈도를 사용하여 모델을 훈련시키기 위해 `use_idf=False, norm=None`으로 지정
- 로지스틱 회귀 분류기는 `penalty` 매개변수를 통해 L1과 L2 규제를 적용하고 규제 매개변수 `C`에 여러 값을 지정해서 규제 강도를 비교

In [ ]:
print('최적의 매개변수 조합: %s ' % gs_lr_tfidf.best_params_)

최적의 매개변수 조합: {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x000001F8D0FBFD30>} 


**네이버 영화 리뷰 감성 분류**

- [데이터셋]('https://github.com/e9t/nsmc)을 사용하여 한글 문장을 감성 분류하는 예제
- 해당 데이터는 네이버 영화 사이트에 있는 리뷰 20만 개를 모은 것
- 네이버 영화 리뷰 데이터셋 깃허브에서 직접 데이터를 내려 받아도 되지만, 편의를 위해 책의 깃허브 ch08 폴더에 데이터셋을 사용
- 20만 개의 데이터 중 15만 개는 훈련 데이터 셋으로 `ratings_train.txt` 파일에 저장되어 잇고 5만 개는 테스트 데이터셋으로 `ratings_test.txt` 파일에 저장
- 리뷰 길이는 140을 넘지 않음
- 부정 리뷰는 1~4까지 점수를 매긴 리뷰, 긍정 리뷰는 6~10까지 점수를 매긴 리뷰 (긍정 부정 리뷰는 약 50%)
- 한글은 영와 달리 조사와 어미가 발달해 있기 대문에 BoW나 어간 추출보다 표제어 추출 방식이 적합
- 이런 작업을 형태소 분석이라 함
- 한글 형태소 분석을 위한 대표적인 패키지는 `konlpy`와 `soynlp`

In [30]:
import konlpy
import pandas as pd
import numpy as np

df_train = pd.read_csv('./datasets/nsmc/ratings_train.txt', delimiter='\t', keep_default_na=False)
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [31]:
X_train = df_train['document'].values
y_train = df_train['label'].values

In [32]:
df_test = pd.read_csv('./datasets/nsmc/ratings_test.txt', delimiter='\t', keep_default_na=False)
X_test = df_test['document'].values
y_test = df_test['label'].values

In [33]:
print(len(X_train), np.bincount(y_train))
print(len(X_test), np.bincount(y_test))

150000 [75173 74827]
50000 [24827 25173]


In [34]:
from konlpy.tag import Okt

okt = Okt()
print(X_train[4])
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


- `TfidfVectorzier`는 기본적으로 공백을 기준으로 토큰을 구분하지만 `tokenizer` 매개변수에 토큰화를 위한 사용자 정의 함수를 전달할 수 있음
- 따라서 앞서 테스트했던 `okt.morphs` 메서드를 전달하면 형태소 분석을 통해 토큰화를 수행할 수 잇음
- `tokenizer` 매개변수를 사용할 때 패턴을 `token_pattern=None`으로 지정하여 `token_pattern` 매개변수가 사용되지 않는다는 경고 메시지가 나오지 않게 함
- `TfidfVectorizer`를 `n_gram_range=(1,2)`로 설정하여 유니그램과 바이그램으로 사용하고 `min_df=3`으로 지정하여 3회 미만으로 등장하는 토큰은 무시
- 또한 `max_df=.9`로 두어 가장 많이 등장하는 상위 10% 토큰도 무시

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=.9, tokenizer=okt.morphs, token_pattern=None)
tfidf.fit(X_train)
X_train_okt = tfidf.transform(X_train)
X_test_okt = tfidf.transform(X_test)

- 탐색할 `SGDClassifier`의 매개변수는 규제를 위한 `alpha` 매개변수
- `RandomizedSearchCV` 클래스를 사용하기 위해 `loguniform` 함수로 탐색 범위를 지정
- 여기서는 `SGDClassifier`의 손실 함수로 로지스틱 손실을 사용하지만, 다른 손실 함수를 매개변수 탐색에 포함할 수 있음
- 총 반복 횟수(n_iter)는 50회로 지정
- 만약 CPU 코어가 여러 개라면 `n_jobs` 매개변수를 지정해서 수행 속도를 높일 수 있음

In [36]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

sgd = SGDClassifier(loss='log', random_state=1)
param_dist = {'alpha': loguniform(0.0001, 100.0)}
rsv_okt = RandomizedSearchCV(estimator=sgd, param_distributions=param_dist, n_iter=50, random_state=1, verbose=1, n_jobs=-1)
rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC351B1C40>},
                   random_state=1, verbose=1)

In [37]:
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

0.8251533333333334
{'alpha': 0.0001001581395585897}


In [38]:
from soynlp.tokenizer import LTokenizer # LTokenizer, MaxScoreTokenizer, RegexTokenizer 가 있음

lto = LTokenizer()
print(lto.tokenize(X_train[4]))

['사이몬페그의', '익살스런', '연기가', '돋보였던', '영화!스파이더맨에서', '늙어보이기만', '했던', '커스틴', '던스트가', '너무나도', '이뻐보였다']


In [39]:
from soynlp.word import WordExtractor

word_ext = WordExtractor()
word_ext.train(X_train)
scores=word_ext.word_scores()

training was done. used memory 1.321 Gbse memory 1.279 Gb
all cohesion probabilities was computed. # words = 85683
all branching entropies was computed # words = 101540
all accessor variety was computed # words = 101540


In [40]:
import math

score_dict = {key: scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy) for key in scores}

In [41]:
lto = LTokenizer(scores=score_dict)
print(lto.tokenize(X_train[4]))

['사이', '몬페그의', '익살스', '런', '연기', '가', '돋보', '였던', '영화', '!스파이더맨에서', '늙어', '보이기만', '했던', '커스틴', '던스트가', '너무', '나도', '이뻐', '보였다']


In [42]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=.9, tokenizer=lto.tokenize, token_pattern=None)
tfidf.fit(X_train)
X_train_soy = tfidf.transform(X_train)
X_test_soy = tfidf.transform(X_test)

In [43]:
rsv_soy = RandomizedSearchCV(estimator=sgd, param_distributions=param_dist, n_iter=50, random_state=1, verbose=1, n_jobs=-1)
rsv_soy.fit(X_train_soy, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001FC351B1C40>},
                   random_state=1, verbose=1)

In [44]:
print(rsv_soy.best_score_)
print(rsv_soy.best_params_)

0.8141066666666665
{'alpha': 0.0001001581395585897}


In [45]:
rsv_soy.score(X_test_soy, y_test)

0.8085

# 4.대용량 데이터 처리: 온라인 알고리즘과 외부 메모리 학습

- 외부 메모리 학습 : 데이터셋을 작은 배치로 나누어 분류기를 점진적으로 학습시킴
- 확률적 경사 하강법 : 한 번에 샘플 하나를  사용하여 모델의 가중치를 업데이트하는 최적화 알고리즘
- `SGDClassifier`의 `partial_fit` 메서드를 사용하여 로지스틱 회귀 모델을 훈련
- 로컬 디스크에서 문서를 직접 읽어 작은 크기의 미니 배치로 만듦

In [75]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words("english")
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower())+' '.join(emoticons).replace('-',''))
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path): # 한번에 문서 하나씩 읽어서 반환하는 stream_docs 제너레이터 함수
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [76]:
next(stream_docs(path='./datasets/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [77]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        pass
    return docs, y

- 외부 메모리 학습에 `CountVectorizer` 클래스를 사용할 수 없음
- 이 클래스는 전체 어휘 사전을 메모리에 가지고 있어야 하기 때문
- 또한 `TfidfVectorizer` 클래스는 역문서 빈도를 계산하기 위해 훈련 데이터셋의 특성 벡터를 모두 메모리에 가지고 있어야 함
- 사이킷런에서 텍스트 처리에 사용할 수 있는 다른 유용한 클래스는 `HashingVectorizer`
- 이는 데이터 종류에 상관없이 사용할 수 있으며, 오스틴 애플비가 만든 32bit MurmurHash3 함수를 사용한 해싱 트릭을 사용함

In [78]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='./datasets/movie_data.csv')

- `tokenizer` 함수와 특성 개수를 2**21로 설정하여 `HasingVectorizer` 클래스를 초기화
- 또 `SGDClassifier` 클래스의 `loss` 매개변수를 `'log'`로 지정하여 로지스틱 회귀 모델로 초기화
- `HashingVectorizer`에서 특성 개수를 크게 하면 해시 충돌 가능성을 줄일 수 있지만, 로지스틱 회귀 모델의 가중치 개수도 늘어남

In [79]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train =get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


In [80]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도 : %.3f' % clf.score(X_test, y_test))

정확도 : 0.868


In [81]:
clf = clf.partial_fit(X_test, y_test)

In [82]:
import pickle
import os

dest = os.path.join('./etc_files/movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

# 5.잠재 디리클레 할등을 사용한 토픽 모델링

- 토픽 모델링은 레이블이 없는 텍스트 문서에 토픽을 할당하는 광범위한 분야
- 인기 있는 토픽 모델링 기법은 **잠재 디리클레 할당**<span style='color:lightgray'>(Latent Dirichlet Allocation, LDA)</span>

## 5.1.LDA를 사용한 텍스트 문서 분해

- LDA의 이면의 수학은 꽤나 복잡하고 베이지안 추론에 관한 지식이 필요
- LDA는 여러 문서에 걸쳐 자주 등장하는 단어의 그룹을 찾는 확률적 생성 모델
- 각 문서를 여러 단어가 혼합된 것으로 가정하면 토픽은 자주 등장하는 단어들로 나타낼 수 있음
- LDA의 입력은 앞서 보았던 BoW 모델
- LDA는 입력 받은 BoW 행렬을 두 개의 행렬로 분해
    > 문서-토픽 행렬 <br>
    > 단어-토픽 행렬
- 이 두 행렬을 곱해서 가능한 작은 오차로 BoW 입력 행렬을 재구성할 수 있도록 LDA가 BoW 행렬을 분해
- 실제로는 LDA가 BoW 행렬에서 찾은 토픽이 관심 대상
- 유일한 단점은  미리 토픽 개수를 정해야 한다는 것
- 즉, 토픽 개수는 LDA의 하이퍼파라미터로 수동으로 지정

## 5.2.사이킷런의 LDA

- 사이킷런의 `LatentDirichletAllocation` 클래스를 사용하여 영화 리부 데이터셋을 분해하고 여러 개의 토픽으로 분류

In [56]:
from sklearn.feature_extraction.text import CountVectorizer as CVect

count = CVect(stop_words='english', max_df=.1, max_features=5000)
X = count.fit_transform(df['review'].values)

- 단어의 최대 문서 빈도를 10%로 지정하여 여러 문서에 걸쳐 너무 자주 등장하는 단어를 제외
- 자주 등장하는 단어를 제외하는 이유는 모든 문서에 결쳐 등장하는 단어일 수 있고, 그런 단어는 문서의 특정 토픽 카테고리와 관련성이 적기 때문
- 가장 자주 등장하는 단어 5,000개로 단어 수를 제한 (`max_features=5000`)
- 데이터셋의 차원을 제한하여 LDA의 추론 성능 향상
- `max_df=.1`과 `max_features=5000`은 임의로 선택한 하이퍼파라미터이므로 튜닝해보면서 결과를 알 수 있음

- 다음의 코드는 문서에서 열 개의 토픽을 추정하도록 `LatentDirichletAllocation` 추정기를 BoW 행렬에 학습하는 방법을 보여 줌

In [60]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda = LDA(n_components=10, random_state=123, learning_method='batch')
X_topics = lda.fit_transform(X)

- `learning_method='batch`로 설정했으므로 `lda` 추정기가 한 번 반복할 때 가능한 모든 훈련 데이터 (BoW 행렬)를 사용하여 학습됨
- `online` 설정보다 느리지만 더 정확한 결과를 만듦
- `learning_method='online'`으로 설정하는 것은 온라인 학습이나 미니 배치 학습과 비슷함

**기댓값 최대화**

- 사이킷런의 LDA 구현은 기댓값 최대화<span style='color:lightgray'>(Expectation-Maximization,EM)</span> 알고리즘을 사용하여 반복적으로 파라미터 추정 값을 업데이트

In [61]:
lda.components_.shape # lda 객체의 components_ 속성에 열 개의 토픽에 대해 오름차순으로 단어(여기서는 5,000개)의 중요도를 담은 행렬이 저장

(10, 5000)

- 결과를 분석하기 위해 열 개의 토픽에서 가장 중요한 단어를 다섯 개씩 출력
- 단어 중요도는 오름차순으로 정렬
- 최상위 다섯 개를 출력하면 토픽 배ㅕㅇㄹ을 역순으로 정렬해야 함

In [63]:
n_top_words =5
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print("토픽 %d" % (topic_idx+1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))

토픽 1
war human beautiful point cinema
토픽 2
comedy kids song fun songs
토픽 3
family mr women wonderful beautiful
토픽 4
series tv dvd book james
토픽 5
documentary footage drug german version
토픽 6
action fun john effects pretty
토픽 7
horror guy night hitchcock sex
토픽 8
series episode police tv episodes
토픽 9
radio american western john chaplin
토픽 10
game horror house fulci italian


- 각 토픽에서 가장 중요한 단어 다섯 개를 기반으로 LDA가 다음 토픽을 구별했다고 추측할 수 있음
    > 1. 대체적으로 형편없는 영화(실제 토픽 카테고리가 되지 못함) <br>
    > 2. 가족 영화 <br>
    > 3. 전쟁 영화 <br>
    > 4. 예술 영화 <br>
    > 5. 범죄 영화 <br>
    > 6. 공포 영화 <br>
    > 7. 코미디 영화 <br>
    > 8. TV 쇼와 관련된 영화 <br>
    > 9. 소설을 원작으로 한 영화 <br>
    > 10. 액션 영화
- 카테고리가 잘 선택되었는지 확인하기 위해 공포 영화 카테고리에서 세 개의 영화 리뷰를 출력

In [65]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx+1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
The Hand of Death aka Countdown in Kung Fu (1976) is a vastly underrated early work by director John Woo. The film stars Dorian Tan (Tan Tao-liang) and features Jackie Chan, Sammo Hung and James Tien in significant supporting roles. Many people believe, or have been lead to believe by deceptive adve ...

공포 영화 #2:
The Hand of Death aka Countdown in Kung Fu (1976) is a vastly underrated early work by director John Woo. The film stars Dorian Tan (Tan Tao-liang) and features Jackie Chan, Sammo Hung and James Tien in significant supporting roles. Many people believe, or have been lead to believe by deceptive adve ...

공포 영화 #3:
Wolfgang Peterson's In the Line of Fire is cunning and occasionally a truly white-knuckled ride, even if once or twice we might feel like we've been down similar roads before. How could one not when Clint Eastwood, right after (allegedly) closing the book on his western legacy, likely closes the one ...
